In [ ]:
import pandas as pd
!pip install transformers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertModel, BertTokenizer
import torch
import progressbar
from transformers import BertTokenizer, TFBertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
f=open("/content/judgement_prediction_test.txt",'r')
df=f.read()
l=df.split('\n')

In [ ]:
import json
tg=open("/content/tag.txt",'r')
dff=tg.read()
lt=[]
dff=eval(dff)
#print(dff)
for i in dff:
  #print(i)
  s=''
  for j in dff[i]:
    s+=j+' '
  lt.append(s)

In [ ]:
print(lt[l.index("1988_291.txt")])

evidence expert witness riot witness homicide murder 


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")
import torch

# Load the Longformer model and tokenizer
'''model_name = 'allenai/longformer-base-4096'
tokenizer = LongformerTokenizer.from_pretrained(model_name)
model = LongformerModel.from_pretrained(model_name)'''

Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


"model_name = 'allenai/longformer-base-4096'\ntokenizer = LongformerTokenizer.from_pretrained(model_name)\nmodel = LongformerModel.from_pretrained(model_name)"

In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import numpy as np
def generate_np_files_for_training(text, tokenizer):
  all_input_ids, all_att_masks, all_labels = [], [], []
  toks = tokenizer.tokenize(text)
  if(len(toks) > 10000):
      toks = toks[len(toks)-10000:]
  tokens = ['[CLS]'] + toks[-510:] + ['[SEP]']
  input_ids = tokenizer.convert_tokens_to_ids(tokens)
  input_tensor = torch.tensor(input_ids)
  # Get the model's output
  with torch.no_grad():
    output = model(input_tensor.unsqueeze(0))
  embedding = output.last_hidden_state[:, 0, :]
  embedded=embedding.tolist()
  return embedded[0]

In [ ]:
def test(dataf):
  embedded_list=[]
  for i in progressbar.progressbar(range(len(dataf['text']))):
    text = dataf['text'].iloc[i]
    k=generate_np_files_for_training(text, tokenizer)
    embedded_list.append(k)
  return embedded_list


In [ ]:
lst=test(train_set)
X=np.array(lst)
np.save("/content/drive/MyDrive/experiment/longformertraining.npy", X)

 95% (4856 of 5082) |################### | Elapsed Time: 6:10:01 ETA:   0:15:06

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/ILDC_single.csv')
train_set= df.query(" split=='train' ")
test_set= df.query(" split=='test' ")

In [ ]:
lst=test_set["name"].tolist()
lst1,lst2,lst3=[],[],[]
for i in l:
  set,set1=[],[]
  if i in lst:
    set=generate_np_files_for_training(test_set.loc[5082+lst.index(i)].at["text"],tokenizer)
    set1=generate_np_files_for_training(test_set.loc[5082+lst.index(i)].at["text"]+lt[l.index(i)],tokenizer)
    #set.extend(generate_np_files_for_training(lt[l.index(i)],tokenizer))
    lst2.append(int(test_set.loc[5082+lst.index(i)].at["label"]))
    lst1.append(set)
    lst3.append(set1)
tst=np.array(lst1)
tst2=np.array(lst3)
#np.save("/content/drive/MyDrive/experiment/longtest.npy",tst)
#np.save("/content/drive/MyDrive/experiment/longtestannotated",tst2)

Token indices sequence length is longer than the specified maximum sequence length for this model (10702 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
lst=test_set["name"].tolist()
print(len(lst))
lst1,lst2,lst3=[],[],[]
set,set1=[],[]
if "1981_187.txt" in lst:
  set=generate_np_files_for_training(test_set.loc[5082+lst.index("1981_187.txt")].at["text"],tokenizer)
  set1=generate_np_files_for_training(test_set.loc[5082+lst.index("1981_187.txt")].at["text"]+lt[l.index("1981_187.txt")],tokenizer)
#set.extend(generate_np_files_for_training(lt[l.index(i)],tokenizer))


1517
['[CLS]', 'and', 'the', 'appellant', 'assaulted', 'the', 'deceased', 'with', 'the', 'blunt', 'side', 'of', 'the', 'gan', '##d', '##hala', 'on', 'the', 'head', 'in', 'the', 'heat', 'of', 'the', 'moment', '.', 'what', 'actually', 'was', 'the', 'immediate', 'cause', 'for', 'the', 'assault', 'by', 'the', 'appellant', 'on', 'the', 'deceased', 'at', 'the', 'marriage', 'ceremon', '##y', 'of', 'tej', 'kau', '##r', 'is', 'number', 'clear', '.', 'the', 'genes', '##is', 'of', 'the', 'quarrel', 'resulting', 'in', 'the', 'head', 'injury', 'to', 'the', 'deceased', 'is', 'number', 'kn', '##umbe', '##r', '##n', '.', 'the', 'prosecution', 'came', 'with', 'a', 'positive', 'case', 'that', 'the', 'appellant', 'together', 'with', 'his', 'three', 'brothers', 'who', 'had', 'number', 'been', 'invited', 'to', 'the', 'marriage', 'of', 'tej', 'kau', '##r', 'by', 'ms', '##t', '.', 'dal', '##i', '##p', 'kau', '##r', 'at', 'the', 'instigation', 'of', 'deceased', 'chan', '##an', 'singh', 'came', 'armed', 'with'

In [ ]:
import numpy as np
tr=np.load("/content/drive/MyDrive/experiment/legaltraining.npy",allow_pickle=True)

In [ ]:
def Listing(a):
  l=[]
  for i in a:
    x=i.tolist()
    l.append(x)
  return l

In [ ]:
def padding_zeros(a):
  for i in a:
    i.extend([0]*(768))

In [ ]:
train=Listing(tr)
padding_zeros(train)
trn=np.array(train)
tst=np.array(lst1)
print(len(tst[0]))

768


In [ ]:
trlabel = train_set['label'].to_numpy().astype('int')
tst1=np.array(lst2)
tst2=np.array(lst3)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
clf = LogisticRegression()
clf.fit(tr,trlabel)
t_label=clf.predict(tst2)
t1_label=clf.predict(tst)
acc=accuracy_score(tst1,t1_label)
acc1=accuracy_score(tst1,t_label)
print(acc,acc1)

0.5957446808510638 0.5851063829787234


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
clf=SVC(kernel="linear")
clf.fit(tr,trlabel)
t_label=clf.predict(tst)
t1_label=clf.predict(tst2)
acc=accuracy_score(tst1,t_label)
acc1=accuracy_score(tst1,t1_label)
print(acc,acc1)

0.5638297872340425 0.574468085106383


In [ ]:
import json
import pandas as pd
tg=open("/content/tag_600_docs.txt",'r')
dff=tg.read()
lt=[]
dff=eval(dff)
lst=list(dff.keys())
for i in dff:
  s=''
  for j in dff[i]:
    s+=' '+j
  lt.append(s)
df=pd.read_csv("test3.csv")
l=df["name"].tolist()
l1=[]
for i in l:
  path=str(i)+".txt"
  if path in lst:
    st=df.iloc[l.index(i)]["text"]+lt[lst.index(path)]
    l1.append(st)
df.insert(1,"appended_text",l1,True)
df.to_csv("append_test_3.csv",index=False)